In [ ]:
#to-do list:
#add wav to mp3 function to audio recorder interface 
#fix an error where uploading multiple files and one files is corrupt an error arise
#change to way of how function work so that it track the exact file it wirks with insted of working with what inside temp
#add introudaction tab and a name for the project



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title install requiered lib
!python -m pip install encodec
!pip install gradio
!pip install torchaudio==0.11.0
!pip install pydub
!git clone https://github.com/facebookresearch/encodec.git
%cd /content/encodec

In [15]:
#@title functions
import gradio as gr
import time as t
import shutil
import subprocess
from scipy.io.wavfile import write
import os
import numpy as np
from datetime import datetime
from os import path
import re
from pydub import AudioSegment
os.makedirs("garbge",exist_ok=True)
os.makedirs("input/temp",exist_ok=True)
os.makedirs("compressed/temp",exist_ok=True)
os.makedirs('decompressed/temp',exist_ok=True)
def compress(audio):
  try:
    print(audio[0].name)
  except:
     
    np.save("garbge/1.npy",np.zeros((512,512)))
    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy'
  sounds=['mp3','wav','ogg']
  print("done uploading")
  #file_name=audio.orig_name
  input_files_sizes=[]
  file_name=[]
  for i in audio:
    names=""
    selfix=i.orig_name.split(".")[-1]

    names=re.sub('[^a-zA-Z1-9]+', '', i.orig_name.split(".")[0])
    names=names+"."+selfix
    print(names,names.split(".")[-1], names.split(".")[-1] in sounds)
    if not names.split(".")[-1] in sounds:
        np.save('garbge/1.npy',np.zeros((512,512)))

        return f'The file {names} is not combaiable','garbge/1.npy'
    file_size = os.path.getsize(i.name)
    input_files_sizes.append(file_size)
    shutil.copyfile(i.name ,"input/temp/"+ names)
    shutil.copyfile(i.name ,"input/"+ names)
    file_name.append(names)
  text=""""""
  print("p1")
  input_data=os.listdir('input/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  output_ecdc=[ i.split(".")[0] for i in input_data ]

  for i , j in zip(input_data,output_ecdc):
      text=text+f"""python -m encodec input/temp/{i} compressed/temp/compressed_{j}.ecdc -f
  """
  t1=t.time()
  print(text,output_ecdc,input_data)
  print("started compressing")
  curent_st=subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()
  print('fininshed compression')

  ####


  if curent_st==1:
    np.save('garbge/1.npy',np.zeros((512,512)))


    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy'
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  
  for k in input_data:
    shutil.copyfile('compressed/temp/'+k ,"compressed/"+k)
  input_data=os.listdir('input/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for i in input_data:
    os.remove('input/temp/'+i)
  output_list=[f'compressed/temp/compressed_{i}.ecdc' for i in output_ecdc]
  print(output_list)
  output_files_sizes=[os.path.getsize(f'compressed/temp/compressed_{i}.ecdc') for i in output_ecdc]
  print(output_files_sizes)

  comp_ratio=[float(100-(i/(j/100))) for j,i in zip(input_files_sizes,output_files_sizes) ]
  print(comp_ratio)

  details=""""""
  for i,j,k,m in zip(file_name,input_files_sizes,output_files_sizes,comp_ratio):
    details =details+i+'\n'+f'time to complete the compression {round(t2-t1,2)} s \n  file size before compresion {round(j/1024,2)} kb \n file size after compression {round(k/1024,2)}  kb  \n ratio of space saved {round(m,2) } % \n'
  print(details)
  return details,output_list

def decompress(file_name):
  
  text=""""""
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  if len(input_data)==0:
    np.save("garbge/1.npy",np.zeros((512,512)))
    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy'
  output_ecdc=[ i.split(".")[0] for i in input_data ]

  for i , j in zip(input_data,output_ecdc):
      text=text+f"""python -m encodec compressed/temp/{i} decompressed/temp/decompressed_{j}.wav -f
  """
  t1=t.time()
  cur_st=subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()
  if cur_st==1:
    np.save('garbge/1.npy',np.zeros((512,512)))


    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy'
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass


  # files
  input_data_dec=os.listdir("decompressed/temp")
  try:
    input_data_dec.remove('.ipynb_checkpoints')
  except:
    pass
  for i in   input_data_dec:                                                                       
    src = f"decompressed/temp/{i}"
    dst = f"decompressed/{i.split('.')[0]}.mp3"
    sound = AudioSegment.from_mp3(src)
    sound.export(dst, format="mp3")
  # convert wav to mp3                                                            
  
  for k in input_data:
    os.remove('compressed/temp/'+k)
  output_list=[f'decompressed/decompressed_{i}.mp3' for i in output_ecdc]
  return f'time to complete Decompression {round(t2-t1,2)} s',output_list

def decompress_1(audio):
  try:
    print(audio[0].name)
  except:
     
    np.save("garbge/1.npy",np.zeros((512,512)))
    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy'
  file_name=[]
  for i in audio:
    names=""
    selfix=i.orig_name.split(".")[-1]
    names=re.sub('[^a-zA-Z1-9]+', '', i.orig_name.split(".")[0])
    names=names+"."+selfix
    if not names.split(".")[1]=='ecdc':
      np.save('garbge/1.npy',np.zeros((512,512)))
      return f"This file {i.orig_name} is not compatiable ",'garbge/1.npy'
  
    shutil.copyfile(i.name ,"input/temp/"+ names)
    shutil.copyfile(i.name ,"input/"+names)
    file_name.append(names)
  text=""""""

  input_data=os.listdir('input/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  output_ecdc=[ i.split(".")[0] for i in input_data ]
  print(input_data,output_ecdc)
  for i , j in zip(input_data,output_ecdc):
      text=text+f"""python -m encodec input/temp/{i} decompressed/temp/decompressed_{j}.wav -f
  """
  t1=t.time()
  cur_st=subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()


  if cur_st==1:
    np.save('garbge/1.npy',np.zeros((512,512)))


    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy'
  input_data=os.listdir('input/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for k in input_data:
    os.remove('input/temp/'+k)
  input_data_dec=os.listdir('decompressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for i in   input_data_dec:                                                                       
    src = f"decompressed/temp/{i}"
    dst = f"decompressed/{i.split('.')[0]}.mp3"
    sound = AudioSegment.from_mp3(src)
    sound.export(dst, format="mp3")
  # convert wav to mp3                                                            
  
 
  output_list=[f'decompressed/decompressed_{i}.mp3' for i in output_ecdc]
  return f'time to complete compression {round(t2-t1,2)} s',output_list




def compress_play(audio):
  try:
    print(audio.name)
  except:
     
    np.save("garbge/1.npy",np.zeros((512,512)))
    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy'
  sounds=['mp3','wav','ogg']
  print("done uploading")
  names=""
  selfix=audio.orig_name.split(".")[-1]
  names=re.sub('[^a-zA-Z1-9]+', '', audio.orig_name.split(".")[0])
  names=names+"."+selfix
  if not names.split(".")[-1] in sounds:
        np.save('garbge/1.npy',np.zeros((512,512)))

        return f'The file {names} is not combaiable','garbge/1.npy'
  file_name=[]
  input_files_sizes=[os.path.getsize(audio.name)]
  shutil.copyfile(audio.name ,"input/temp/"+ names)
  shutil.copyfile(audio.name ,"input/"+ names)
  file_name.append(names)
  text=""""""
  
  input_data=os.listdir('input/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  output_ecdc=[ i.split(".")[0] for i in input_data ]

  for i , j in zip(input_data,output_ecdc):
      text=text+f"""python -m encodec input/temp/{i} compressed/temp/compressed_{j}.ecdc -f
  """
  
  print('started compressing')
  t1=t.time()
  print(text,output_ecdc,input_data)
  cur_st=subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()
  print("done compressing")
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass

  if cur_st==1:
    np.save('garbge/1.npy',np.zeros((512,512)))


    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy'
  ####
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for k in input_data:
    shutil.copyfile('compressed/temp/'+k ,"compressed/"+k)
  input_data=os.listdir('input/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for i in input_data:
    os.remove('input/temp/'+i)
  output_list=[f'compressed/temp/compressed_{i}.ecdc' for i in output_ecdc]
  output_files_sizes=[os.path.getsize(f'compressed/temp/compressed_{i}.ecdc') for i in output_ecdc]
  details=""""""
  comp_ratio=[float(100-(i/(j/100))) for j,i in zip(input_files_sizes,output_files_sizes) ]
  for i,j,k,m in zip(file_name,input_files_sizes,output_files_sizes,comp_ratio):
    details =details+i+'\n'+f'time to complete the compression {round(t2-t1,2)} s \n  file size before compresion {round(j/1024,2)} kb \n file size after compression {round(k/1024,2)}  kb  \n ratio of space saved {round(m,2) } % \n'

  return details,output_list

def decompress_play(file_name):
 
  text=""""""
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass

  if len(input_data)==0:
    np.save('garbge/1.npy',np.zeros((512,512)))


    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy','garbge/1.npy'
  output_ecdc=[ i.split(".")[0] for i in input_data ]

  for i , j in zip(input_data,output_ecdc):
      text=text+f"""python -m encodec compressed/temp/{i} decompressed/temp/decompressed_{j}.wav -f
  """
  t1=t.time()
  cur_st=subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for k in input_data:
    os.remove('compressed/temp/'+k)

  input_data_dec=os.listdir("decompressed/temp")
  try:
    input_data_dec.remove('.ipynb_checkpoints')
  except:
    pass
  for i in   input_data_dec:                                                                       
    src = f"decompressed/temp/{i}"
    dst = f"decompressed/{i.split('.')[0]}.mp3"
    sound = AudioSegment.from_mp3(src)
    sound.export(dst, format="mp3")
  # convert wav to mp3                                                            
  


  output_list=[f'decompressed/decompressed_{i}.mp3' for i in output_ecdc]
  
  return f'time to complete decompression {round(t2-t1,2)} s',output_list[0],output_list[0]

def decompress_1_play(audio):
  try:
    print(audio.name)
  except:
     
    np.save("garbge/1.npy",np.zeros((512,512)))
    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy','garbge/1.npy'
  sounds=['mp3','wav','ogg']
  print("done uploading")
  names=""
  selfix=audio.orig_name.split(".")[-1]
  names=re.sub('[^a-zA-Z1-9]+', '', audio.orig_name.split(".")[0])
  names=names+"."+selfix
  if not names.split(".")[1]=='ecdc':  
        np.save('garbge/1.npy',np.zeros((512,512)))

        return f'The file {names} is not combaiable','garbge/1.npy','garbge/1.npy'
  file_name=[]

  

  shutil.copyfile(audio.name ,"input/temp/"+ names)
  shutil.copyfile(audio.name ,"input/"+ names)
  file_name.append(names)
  text=""""""

  input_data=os.listdir('input/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  output_ecdc=[ i.split(".")[0] for i in input_data ]
  print(input_data,output_ecdc)
  for i , j in zip(input_data,output_ecdc):
      text=text+f"""python -m encodec input/temp/{i} decompressed/temp/decompressed_{j}.wav -f
  """
  t1=t.time()
  cur_st= subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()
  if cur_st==1:
    np.save('garbge/1.npy',np.zeros((512,512)))


    return 'The operation failed for unkonwn or corrupt input type','garbge/1.npy','garbge/1.npy'
  input_data=os.listdir('input/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for k in input_data:
    os.remove('input/temp/'+k)
  input_data_dec=os.listdir("decompressed/temp")
  try:
    input_data_dec.remove('.ipynb_checkpoints')
  except:
    pass

  for i in   input_data_dec:                                                                       
    src = f"decompressed/temp/{i}"
    dst = f"decompressed/{i.split('.')[0]}.mp3"
    sound = AudioSegment.from_mp3(src)
    sound.export(dst, format="mp3")
  # convert wav to mp3                                                            
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for k in input_data:
    os.remove('compressed/temp/'+k)
  output_list=[f'decompressed/decompressed_{i}.mp3' for i in output_ecdc]

  return f'time to complete decompression {round(t2-t1,2)} s',output_list[0],output_list[0]
def large_comp(audio):

  print("done uploading")
  #file_name=audio.orig_name
  selfix=audio.split(".")[-1]
  names=""
  names=re.sub('[^a-zA-Z1-9]+', '', audio.split(".")[0])
  names=names+"."+selfix
  file_name=[names]
  shutil.copyfile(f"/content/drive/MyDrive/sounds/{audio}","input/temp/"+ f'{names}')
  shutil.copyfile("input/temp/"+ f'{names}',"input/"+ f'{names}')
  input_files_sizes=[os.path.getsize(f"input/temp/{names}")]

  text=""""""
  
  input_data=file_name
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  output_ecdc=[ i.split(".")[0] for i in input_data ]

  for i , j in zip(input_data,output_ecdc):
      text=text+f"""python -m encodec input/temp/{i} compressed/temp/compressed_{j}.ecdc -f
  """
  print('started compressing')
  t1=t.time()
  curt_st=subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()
  print("done compressing")
  if curt_st==1:
    np.save('garbge/1.npy',np.zeros((512,512)))


    return "",'The operation failed for unkonwn or corrupt input type','garbge/1.npy','garbge/1.npy','garbge/1.npy'
  ####
  input_data=[i for i in os.listdir('compressed/temp') if i!=".ipynb_checkpoints"]
 
  for k in input_data:
    shutil.copyfile('compressed/temp/'+k ,"compressed/"+k)
  input_data=os.listdir('input/temp')
  try:
      input_data.remove('.ipynb_checkpoints')
  except:
      pass
  for i in input_data:
    os.remove('input/temp/'+i)
  output_list_COMP=[f'compressed/compressed_{i}.ecdc' for i in output_ecdc]
  output_files_sizes=[os.path.getsize(f'compressed/temp/compressed_{i}.ecdc') for i in output_ecdc]
  details=""""""
  comp_ratio=[float(100-(i/(j/100))) for j,i in zip(input_files_sizes,output_files_sizes) ]
  for i,j,k,m in zip(file_name,input_files_sizes,output_files_sizes,comp_ratio):
    details =details+i+'\n'+f'time to complete the compression {round(t2-t1,2)} s \n  file size before compresion {round(j/1024,2)} kb \n file size after compression {round(k/1024,2)}  kb  \n ratio of space saved {round(m,2) } % \n'
  text=""""""


  for i , j in zip(output_ecdc,output_ecdc):
      text=text+f"""python -m encodec compressed/temp/compressed_{i}.ecdc decompressed/temp/decompressed_{j}.wav -f
  """
  t1=t.time()
  print('started_decompresings')
  curt_st=subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()
  if curt_st==1:
    np.save('garbge/1.npy',np.zeros((512,512)))


    return "",'The operation failed for unkonwn or corrupt input type','garbge/1.npy','garbge/1.npy','garbge/1.npy'
  print('done decompressing')
  input_data=os.listdir('compressed/temp')
  print(input_data)
  input_data_dec=os.listdir('decompressed/temp')
  try:
    input_data_dec.remove('.ipynb_checkpoints')
  except:
    pass
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  print(input_data)
  for i in   input_data_dec:                                                                       
    src = f"decompressed/temp/{i}"
    dst = f"decompressed/{i.split('.')[0]}.mp3"
    sound = AudioSegment.from_mp3(src)
    sound.export(dst, format="mp3")
  print(input_data)

  for k in input_data:
      os.remove('compressed/temp/'+k)

  output_list=[f'decompressed/decompressed_{i}.mp3' for i in output_ecdc]
  print(output_list)
  print(details)
  print(f'time to complete decompression {round(t2-t1,2)} s')
  print(output_list_COMP[0])
  print(output_list[0])

  return details,f'time to complete decompression {round(t2-t1,2)} s',output_list_COMP[0],output_list[0],output_list[0]



In [16]:
#@title interface
accepted_selfixs=['mp3','wav','ogg','m4a']
sound_files=[i for i in os.listdir('/content/drive/MyDrive/sounds') if i.split(".")[1] in accepted_selfixs]


with gr.Blocks() as demo:
  
  gr.Markdown("""<center>   <span style="font-family:Times New Roman; color:red;font-size:4em;">         Audio compression using neural networks</span>
                """)
        

  with gr.Tab(" Compress And Decompress "):
    with gr.Row():
        with gr.Column():
            input = gr.File(label="insert multiple  files",file_count='multiple')
        with gr.Column():
            comp = gr.Button(value="Compress")
            decomp = gr.Button(value="DeCompress")
    with gr.Row():
            output_text_com = gr.Textbox(label="Compression details")
    with gr.Row():
            output_text_decom = gr.Textbox(label="DeCompression details")
    with gr.Row():
        with gr.Column():
            output_file=gr.File(label="ecdc file",file_count='multiple')
        with gr.Column():
            output_file_1=gr.File(label="wav file")
    with gr.Row():
        with gr.Column():
            input_1 = gr.File(label="insert file",file_count='multiple')
        with gr.Column():
            decomp_1 = gr.Button(value="DeCompress")
    with gr.Row():
        with gr.Column():
            output_text_1_decom = gr.Textbox(label="DeCompression details")
        with gr.Column():
            output_file_2=gr.File(label="wav file")
    
  with gr.Tab(" Compress And Decompress and play "):
    with gr.Row():
        with gr.Column():
            input_play = gr.File(label="insert file",file_count='single')
        with gr.Column():
            comp_play = gr.Button(value="Compress")
            decomp_play = gr.Button(value="DeCompress")
    with gr.Row():
        

            output_text_play_com = gr.Textbox(label="Compression details")
    with gr.Row():
        

            output_text_play_decom = gr.Textbox(label="DeCompression details")
    with gr.Row():
        with gr.Column():
            output_file_play=gr.File(label="ecdc file")
        with gr.Column():
            output_file_1_play=gr.Audio(label="wav file")
            output_file_11_play=gr.File(label='wav file')
    with gr.Row():
        with gr.Column():
            input_1_play = gr.File(label="insert file",file_count='single')
        with gr.Column():
            decomp_1_play = gr.Button(value="DeCompress")
    with gr.Row():
        with gr.Column():
            output_text_1_play_decom = gr.Textbox(label="Decompression Details")
        with gr.Column():
            output_file_2_play=gr.Audio(label="wav file")    
            output_file_22_play=gr.File(label="wav file")    

  with gr.Tab("large files"):
    with gr.Row():
      with gr.Column():
        large_input=gr.Dropdown(sound_files)
      with gr.Column():
        large_comp1=gr.Button(label='compress files')
    with gr.Row():
      l_text=gr.Textbox(label="Compression details")
      l_text_2=gr.Textbox(label="DeCompression details")
      l_file=gr.File(label='ecdc file')
      l_wav_file=gr.Audio(label='wav file')
      l_file_file=gr.File(label='wav file')
    comp.click(compress, inputs=input, outputs=[output_text_com,output_file] )
    decomp.click(decompress, inputs=output_file, outputs=[output_text_decom,output_file_1] )
    decomp_1.click(decompress_1, inputs=input_1, outputs=[output_text_1_decom,output_file_2] )

    comp_play.click(compress_play, inputs=input_play, outputs=[output_text_play_com,output_file_play] )
    decomp_play.click(decompress_play, inputs=output_file_play, outputs=[output_text_play_decom,output_file_1_play,output_file_11_play] )
    decomp_1_play.click(decompress_1_play, inputs=input_1_play, outputs=[output_text_1_play_decom,output_file_2_play,output_file_22_play] )
    large_comp1.click(large_comp,inputs=large_input,outputs=[l_text,l_text_2,l_file,l_wav_file,l_file_file])

demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/tmp/setuper3lxe4o.cfg
done uploading
/tmp/architecture7u_sjgvy.ecdc
done uploading
['architecture.ecdc'] ['architecture']
/tmp/setup_z1mkmv3.cfg
done uploading
/tmp/architecturezhcq1atd.wav
done uploading
started compressing
python -m encodec input/temp/architecture.ecdc compressed/temp/compressed_architecture.ecdc -f
  python -m encodec input/temp/architecture.wav compressed/temp/compressed_architecture.ecdc -f
   ['architecture', 'architecture'] ['architecture.ecdc', 'architecture.wav']
done compressing
Keyboard interruption in main thread... closing server.


In [ ]:
#@title record audio interface 
import gradio as gr
import numpy as np
from scipy.io.wavfile import write
from datetime import datetime
import gradio as gr
import time as t
import shutil
import subprocess
from scipy.io.wavfile import write
import os
import numpy as np
from datetime import datetime
os.makedirs("input/temp",exist_ok=True)
os.makedirs("compressed/temp",exist_ok=True)
os.makedirs('decompressed',exist_ok=True)
def reverse_audio(audio):
  sr,data=audio
  now = datetime.now()

  current_time = now.strftime("%H:%M:%S")
  temp_name=str(current_time).replace(":","_")
  write(f"input/temp/record_{temp_name}.wav", sr, data.astype(np.int32))
  print("done uploading")
  #file_name=audio.orig_name
  file_name=[f'record_{temp_name}.wav']
  input_files_sizes=[os.path.getsize(f"input/temp/record_{temp_name}.wav")]
  shutil.copyfile(f"input/temp/record_{temp_name}.wav","input/"+ f'record_{temp_name}.wav')
  text=""""""
  
  input_data=os.listdir('input/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  output_ecdc=[ i.split(".")[0] for i in input_data ]

  for i , j in zip(input_data,output_ecdc):
      text=text+f"""python -m encodec input/temp/{i} compressed/temp/compressed_{j}.ecdc -f
  """
  print('started compressing')
  t1=t.time()
  subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()
  print("done compressing")
  
  ####
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for k in input_data:
    shutil.copyfile('compressed/temp/'+k ,"compressed/"+k)
  input_data=os.listdir('input/temp')

  for i in input_data:
    os.remove('input/temp/'+i)
  output_list_COMP=[f'compressed/compressed_{i}.ecdc' for i in output_ecdc]
  output_files_sizes=[os.path.getsize(f'compressed/temp/compressed_{i}.ecdc') for i in output_ecdc]
  details=""""""
  comp_ratio=[float(100-(j/(i/100))) for j,i in zip(input_files_sizes,output_files_sizes) ]
  for i,j,k,m in zip(file_name,input_files_sizes,output_files_sizes,comp_ratio):
    details =details+i+'\n'+f'time to complete the compression {round(t2-t1,2)} s \n  file size before compresion {round(j/1024,2)} kb \n file size after compression {round(k/1024,2)}  kb  \n ratio of space saved {round(m,2) } % \n'
  text=""""""
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  output_ecdc=[ i.split(".")[0] for i in input_data ]

  for i , j in zip(input_data,output_ecdc):
      text=text+f"""python -m encodec compressed/temp/{i} decompressed/decompressed_{j}.wav -f
  """
  t1=t.time()
  subprocess.call(text, shell=True,stderr=subprocess.STDOUT)
  t2=t.time()
  input_data=os.listdir('compressed/temp')
  try:
    input_data.remove('.ipynb_checkpoints')
  except:
    pass
  for k in input_data:
    os.remove('compressed/temp/'+k)
  output_list=[f'decompressed/decompressed_{i}.wav' for i in output_ecdc]
  
   

  return details,f'time to complete decompression {round(t2-t1,2)} s',output_list_COMP[0],output_list[0]

demo = gr.Interface(fn=reverse_audio, 
                    inputs="microphone", 
                    outputs=['text','text','file',"audio"] )
demo.launch(debug=True)

In [ ]:
!pip3 freeze > requirements.txt